In [1]:
import numpy as np
import pandas as pd
from random import*
import mdptoolbox
import time

In [2]:
#Transformation of data set into DataFrame, into np.matrix, without headers or other stuff.
def Prepared_Matrix(csv):
    _ = pd.read_csv(csv, header=None)
    _ = _.values
    _ = np.delete(_,0,1)
    _ = np.delete(_,0,0)
    _ = _.astype(float)
    _ = np.asmatrix(_)
    return _

In [3]:
#If you want to create one mean matrix using various preapared matrices.
def Mean_Matrix(M1, M2, M3=None, M4=None, M5=None, n=2):
    matrices = [M1, M2, M3, M4, M5]
    return sum(matrices)/n

In [4]:
#If you want to reduce the dimensions of the original matrix.
def nxn(M, desired_n):
    rows = np.shape(M)[0]
    cols = np.shape(M)[1]
    if (rows == cols) & rows%desired_n == 0:
        _ = np.empty((desired_n,desired_n), dtype=int)
        n = int(rows/desired_n)
        for ii in range(int(desired_n)):
            for jj in range(int(desired_n)):
                _[ii,jj] = np.sum(M[ii*n:(ii+1)*n, jj*n:(jj+1)*n])
    else:
        print("Error: non-square matrix, or no exact division")
    return _

In [5]:
#Creates a Markov matrix; a transition matrix.
def Transition_Matrix(M):
    n = np.shape(M)[0]
    _ = np.empty((n,n), dtype=float)
    for ii in range(n):
        sf = np.sum(M[ii,:])
        if sf != 0:
            _[ii,:] = M[ii,:]/(np.sum(M[ii,:]))
        else:
            _[ii,:] = 1/n
    return _

In [6]:
#Creates a reward matrix.
def Reward_Matrix(desired_n, mean_price, std_dev):
    _ = np.zeros((desired_n, desired_n))
    for ii in range(desired_n):
        for jj in range(desired_n):
            _[ii,jj] = normalvariate(mean_price, std_dev)
    return _

In [7]:
t0 = time.time()
P1 = Prepared_Matrix("Demand_Jun_04_2011.csv")
print("One csv took "+str(time.time()-t0)+".")
P2 = Prepared_Matrix("Demand_Jun_05_2011.csv")
P3 = Prepared_Matrix("Demand_Jun_06_2011.csv")
P4 = Prepared_Matrix("Demand_Jun_07_2011.csv")
P5 = Prepared_Matrix("Demand_Jun_08_2011.csv")
print("All csv took "+str(time.time() - t0)+".")

/home/santiagortiz/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


One csv took 13.319872856140137.
All csv took 72.3278112411499.


In [8]:
n1 = nxn(P1, 10)
n2 = nxn(P2, 10)
n3 = nxn(P3, 10)
n4 = nxn(P4, 10)
n5 = nxn(P5, 10)

In [9]:
T1 = Transition_Matrix(n1)
T2 = Transition_Matrix(n2)
T3 = Transition_Matrix(n3)
T4 = Transition_Matrix(n4)
T5 = Transition_Matrix(n5)

In [10]:
R1 = Reward_Matrix(10, 31, 9)
R2 = Reward_Matrix(10, 24, 7)
R3 = Reward_Matrix(10, 40, 1)
R4 = Reward_Matrix(10, 41, 30)
R5 = Reward_Matrix(10, 30, 2)

In [11]:
#Creating the initial policy vector. Required by MDPToolbox.
p0 = np.array([1,1,1,1,1,1,1,1,1,1])

In [12]:
#The np.array that contains the transition matrices. Required by MDPToolbox.
T = np.array([T1, T2, T3, T4, T5])

In [13]:
#The np.array that contains the reward matrices. Required by MDPToolbox.
R = np.array([R1, R2, R3, R4, R5])

In [14]:
#MDPToolbox object. Uses the method PolicyIteration.
pi = mdptoolbox.mdp.PolicyIteration(T, R, 0.4)

In [15]:
pi.run()

In [16]:
#The optimal policy vector.
pi.policy

(2, 3, 3, 2, 3, 3, 2, 2, 2, 3)

In [17]:
#Value function.
pi.V

(71.31754521571456,
 74.41280392042704,
 86.84266159938865,
 74.60110758866284,
 97.96769525931659,
 82.07655675146275,
 72.69954089987955,
 70.06556665016687,
 70.9937498324574,
 80.10988992844396)

In [18]:
#CPU time.
pi.time

0.018876314163208008

In [19]:
pi.iter

1

In [20]:
piRVI = mdptoolbox.mdp.RelativeValueIteration(T, R, 0.4)

In [21]:
piRVI.run()

In [22]:
piRVI.policy

(2, 3, 3, 2, 3, 3, 2, 2, 2, 3)

In [23]:
piRVI.iter

6

In [24]:
piMod = mdptoolbox.mdp.PolicyIterationModified(T, R, 0.4)

In [25]:
piMod.run()

In [27]:
piMod.policy

(2, 3, 3, 2, 3, 3, 2, 2, 2, 3)

In [28]:
piMod.iter

2

In [33]:
piVI = mdptoolbox.mdp.ValueIteration(T, R, 0.4)

In [34]:
piVI.run()

In [35]:
piVI.policy

(2, 3, 3, 2, 3, 3, 2, 2, 2, 3)

In [36]:
piVI.iter

6

In [37]:
piVIGS = mdptoolbox.mdp.ValueIteration(T, R, 0.4)

In [38]:
piVIGS.run()

In [39]:
piVIGS.policy

(2, 3, 3, 2, 3, 3, 2, 2, 2, 3)

In [40]:
piVIGS.iter

6

### Ideas
- Transition matrices could be dynamic. Not quite Markovian dynamics. The present state depends on previous states.
    - LEARNING DEEP MEAN FIELD GAMES FOR MODELING LARGE POPULATION BEHAVIOR. (2018). Jiachen Yang, Xiaojing Ye, Rakshit Trivedi, Huan Xu & Hongyuan Zha.
- Good predictors of the transition matrix are:
    - the transition matrix of the same time one week ago
    - the transition matrix of the previous time bin (a nearest previous neighbour model)
    - reflection of demand directions: if many people go to j, many people must return from j.
    - real time feedback, somehow reinforced.